In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=2b6b76facc4c2d978f723c1655f6eca28f43637cab1a95df5c07d3d486ae715d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 6 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package 

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sem08").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
spark

In [ ]:
# Пробрасываем ngrk - чтобы открыть Spark UI в браузере
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip
!./ngrok authtoken <your-ngrok-token>
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 5
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://d526-34-125-38-109.ngrok-free.app


---

In [ ]:
import pandas as pd

df = pd.DataFrame(
    data=[[1, 2], [2, 12], [3, 6], [4, 25]],
    columns=['a', 'b']
)
df

,a,b
0,1,2
1,2,12
2,3,6
3,4,25


In [ ]:
!rm -rf example_1
!mkdir example_1

df.to_parquet('/content/example_1/part-0000.parquet')

In [ ]:
!pwd

/content


In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.functions import col

spark.read.parquet('/content/example_1') \
     .withColumn('c', f.col('a') + 5) \
     .filter(f.col('b') > 10) \
     .write.parquet('/content/example_1_output')

In [ ]:
spark.read.parquet('/content/example_1') \
     .withColumn('c', f.col('a') + 5) \
     .filter(f.col('b') > 10) \
     .explain(True)

== Parsed Logical Plan ==
'Filter ('b > 10)
+- Project [a#12L, b#13L, (a#12L + cast(5 as bigint)) AS c#16L]
   +- Relation [a#12L,b#13L] parquet

== Analyzed Logical Plan ==
a: bigint, b: bigint, c: bigint
Filter (b#13L > cast(10 as bigint))
+- Project [a#12L, b#13L, (a#12L + cast(5 as bigint)) AS c#16L]
   +- Relation [a#12L,b#13L] parquet

== Optimized Logical Plan ==
Project [a#12L, b#13L, (a#12L + 5) AS c#16L]
+- Filter (isnotnull(b#13L) AND (b#13L > 10))
   +- Relation [a#12L,b#13L] parquet

== Physical Plan ==
*(1) Project [a#12L, b#13L, (a#12L + 5) AS c#16L]
+- *(1) Filter (isnotnull(b#13L) AND (b#13L > 10))
   +- *(1) ColumnarToRow
      +- FileScan parquet [a#12L,b#13L] Batched: true, DataFilters: [isnotnull(b#13L), (b#13L > 10)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_1], PartitionFilters: [], PushedFilters: [IsNotNull(b), GreaterThan(b,10)], ReadSchema: struct<a:bigint,b:bigint>



---

In [ ]:
!rm -rf /content/example_2_output_1
!rm -rf /content/example_2_output_2

df = spark.read.parquet('/content/example_1') \
          .withColumn('c', f.col('a') + 5)

df.filter(f.col('b') > 10).write.parquet('/content/example_2_output_1')
df.filter(f.col('b') <= 10).write.parquet('/content/example_2_output_2')

In [ ]:
!rm -rf /content/example_2_output_1
!rm -rf /content/example_2_output_2

df = spark.read.parquet('/content/example_1') \
          .withColumn('c', f.col('a') + 5) \
          .cache()

df.filter(f.col('b') > 10).write.parquet('/content/example_2_output_1')
df.filter(f.col('b') <= 10).write.parquet('/content/example_2_output_2')

---

In [ ]:
spark.read.parquet('/content/example_1')

DataFrame[a: bigint, b: bigint]

In [ ]:
df = spark.read.parquet('/content/example_1')
df.select('a').explain(True)

== Parsed Logical Plan ==
'Project ['a]
+- Relation [a#190L,b#191L] parquet

== Analyzed Logical Plan ==
a: bigint
Project [a#190L]
+- Relation [a#190L,b#191L] parquet

== Optimized Logical Plan ==
Project [a#190L]
+- Relation [a#190L,b#191L] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [a#190L] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_1], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<a:bigint>



In [ ]:
df = spark.read.parquet('/content/example_1')
df.select(f.count("*")).explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias(count(1), Some(org.apache.spark.sql.Column$$Lambda$4058/1266629882@3b2de5e))]
+- Relation [a#298L,b#299L] parquet

== Analyzed Logical Plan ==
count(1): bigint
Aggregate [count(1) AS count(1)#303L]
+- Relation [a#298L,b#299L] parquet

== Optimized Logical Plan ==
Aggregate [count(1) AS count(1)#303L]
+- Project
   +- Relation [a#298L,b#299L] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(1)], output=[count(1)#303L])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=415]
      +- HashAggregate(keys=[], functions=[partial_count(1)], output=[count#306L])
         +- FileScan parquet [] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_1.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<>



In [ ]:
!mkdir example_3
!mkdir example_3/a=1
!mkdir example_3/a=2

df = pd.DataFrame(
    data=[[1, 2], [1, 12], [1, 6], [1, 25]],
    columns=['a', 'b']
)
df.to_parquet('example_3/a=1/part1.parquet')

df = pd.DataFrame(
    data=[[2, 2], [2, 12], [2, 6], [2, 25]],
    columns=['a', 'b']
)
df.to_parquet('example_3/a=2/part1.parquet')

In [ ]:
df = spark.read.parquet('/content/example_3')
df.filter(f.col('a') == 2).explain(True)

== Parsed Logical Plan ==
'Filter ('a = 2)
+- Relation [a#341,b#342L] parquet

== Analyzed Logical Plan ==
a: int, b: bigint
Filter (a#341 = 2)
+- Relation [a#341,b#342L] parquet

== Optimized Logical Plan ==
Filter (isnotnull(a#341) AND (a#341 = 2))
+- Relation [a#341,b#342L] parquet

== Physical Plan ==
*(1) Project [a#341, b#342L]
+- *(1) ColumnarToRow
   +- FileScan parquet [b#342L,a#341] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_3], PartitionFilters: [isnotnull(a#341), (a#341 = 2)], PushedFilters: [], ReadSchema: struct<b:bigint>



---

In [ ]:
!rm -rf example_4
!mkdir example_4

import uuid
import random
import numpy as np

commands = ['read', 'write', 'other', None]
logins = [str(uuid.uuid4()) for _ in range(100)]

data = [[np.random.choice(commands, p=[0.5, 0.3, 0.15, 0.05]), random.choice(logins)] for _ in range(10000)]
df = pd.DataFrame(data=data, columns=['command', 'login'])
df.to_parquet('example_4/part-0000.parquet')
df.head(10)

,command,login
0,write,d804be23-17f5-49e5-85c6-6e5339b4dd36
1,read,d804be23-17f5-49e5-85c6-6e5339b4dd36
2,other,0252f7b4-95dd-4a2a-8e20-62725cd7f73a
3,read,e6b8bf27-cdf9-4dad-95e8-8ebcc95d778b
4,write,2d0a8167-2062-4000-9850-ba154bcda578
5,read,c85f40c0-dfba-4fb5-98b1-f32ee7cf70f7
6,read,f527817b-c37f-40d2-b9ac-7f487e19415a
7,read,52b789d4-6ca9-4f45-89d9-27766140a949
8,read,1b7dfe41-5f4b-4d10-9faf-d3e2b6983b82
9,write,fb576683-70e7-443b-8163-ffce55fb249d


In [ ]:
df = spark.read.parquet('/content/example_4')

df.groupby('command') \
  .agg(
      f.count('*'),
      f.countDistinct('login')
  ) \
  .repartition(1) \
  .show()

+-------+--------+---------------------+
|command|count(1)|count(DISTINCT login)|
+-------+--------+---------------------+
|   NULL|     507|                   99|
|   read|    5051|                  100|
|  other|    1458|                  100|
|  write|    2984|                  100|
+-------+--------+---------------------+



In [ ]:
spark.conf.get('spark.sql.shuffle.partitions')

'200'

---

In [ ]:
!rm -rf example_5
!mkdir example_5

data = [[login, random.random() < 0.6] for login in logins]
df = pd.DataFrame(data=data, columns=['login', 'is_robot_flg'])
df.to_parquet('example_5/part-0000.parquet')
df

,login,is_robot_flg
0,a6e416d3-579a-444e-b880-293a5ca59a8f,False
1,e40a907d-5123-4d64-bfd4-5c58d33a78e8,True
2,a42341b9-629c-4eeb-a85c-a833a745013d,True
3,c5c65234-2e82-4ce9-9c3f-df5e8076143e,False
4,ec729e2e-b2fc-4654-81b3-7678db0a857b,True
...,...,...
95,3e90d0d5-0071-40f9-8c3c-f5514be37cf0,False
96,fec6f79f-82b1-4c96-b99f-977af6ee03c3,False
97,22e463e3-a538-4886-87e1-01b4cb85ef08,False
98,56fb4968-7ab8-42f9-9600-c9efd8433755,False


In [ ]:
usage = spark.read.parquet('/content/example_4')
staff = spark.read.parquet('/content/example_5')

usage.join(staff, ['login'], 'inner') \
     .groupby('is_robot_flg') \
     .agg( f.count('*'), f.countDistinct('command') ) \
     .repartition(1) \
     .write.parquet('/content/example_5_output')

In [ ]:
usage = spark.read.parquet('/content/example_4')
staff = spark.read.parquet('/content/example_5')

usage.join(staff, ['login'], 'inner') \
     .groupby('is_robot_flg') \
     .agg( f.count('*'), f.countDistinct('command') ) \
     .repartition(1) \
     .show()

+------------+--------+-----------------------+
|is_robot_flg|count(1)|count(DISTINCT command)|
+------------+--------+-----------------------+
|        true|    5883|                      3|
|       false|    4117|                      3|
+------------+--------+-----------------------+



In [ ]:
from pyspark.sql.functions import broadcast

usage = spark.read.parquet('/content/example_4')
staff = spark.read.parquet('/content/example_5')

usage.join(broadcast(staff), ['login'], 'inner') \
     .groupby('is_robot_flg') \
     .agg( f.count('*'), f.countDistinct('command') ) \
     .repartition(1) \
     .show()

+------------+--------+-----------------------+
|is_robot_flg|count(1)|count(DISTINCT command)|
+------------+--------+-----------------------+
|        true|    5883|                      3|
|       false|    4117|                      3|
+------------+--------+-----------------------+



In [ ]:
# before
usage.join(broadcast(staff), ['login'], 'right_outer')

# after
tmp = usage.join(broadcast(staff), ['login'], 'inner')
staff.join(broadcast(tmp), ['login'], 'left_outer')

# right
keys = staff.select('login').distinct()
tmp = usage.join(broadcast(keys), ['login'], 'inner')
staff.join(broadcast(tmp), ['login'], 'left_outer')